# Movilidad y características de comunas


In [25]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
matplotlib.style.use("fivethirtyeight")

## Load Data

In [26]:
prod_33 = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto33/"
mobility = pd.read_csv(prod_33 + "IndiceDeMovilidad.csv")
mobility["Fecha"] = pd.to_datetime(mobility.Fecha, format="%Y-%m-%d")
mobility['lagged_Fecha'] = mobility['Fecha'] - pd.Timedelta("8d")

print("Fecha min: ", mobility.Fecha.min(), " fecha max: ", mobility.Fecha.max())
mobility.columns

Fecha min:  2020-02-26 00:00:00  fecha max:  2020-05-17 00:00:00


Index(['Region', 'Codigo region', 'Comuna', 'Codigo comuna', 'Superficie_km2',
       'Poblacion', 'IM_interno', 'IM_externo', 'IM', 'Fecha', 'lagged_Fecha'],
      dtype='object')

In [27]:
mobility.head()

,Region,Codigo region,Comuna,Codigo comuna,Superficie_km2,Poblacion,IM_interno,IM_externo,IM,Fecha,lagged_Fecha
0,Ñuble,16.0,Chillán,16101,477.58,198624.0,5.922157,1.878393,7.800551,2020-02-26,2020-02-18
1,Ñuble,16.0,Chillán,16101,477.58,198624.0,5.453929,1.877832,7.331762,2020-02-27,2020-02-19
2,Ñuble,16.0,Chillán,16101,477.58,198624.0,5.616649,1.986287,7.602936,2020-02-28,2020-02-20
3,Ñuble,16.0,Chillán,16101,477.58,198624.0,4.800831,1.883542,6.684373,2020-02-29,2020-02-21
4,Ñuble,16.0,Chillán,16101,477.58,198624.0,3.706828,1.611526,5.318354,2020-03-01,2020-02-22


# Data fechas cuarentena

In [28]:
path_cuarentena = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto29/Cuarentenas-Totales.csv"

df_cuarentena = pd.read_csv(path_cuarentena)

df_cuarentena["fecha_inicio"] = pd.to_datetime(df_cuarentena['Fecha de Inicio'], format="%Y-%m-%d %H:%M:%S").dt.date
df_cuarentena["fecha_termino"] = pd.to_datetime(df_cuarentena['Fecha de Término'], format="%Y-%m-%d %H:%M:%S").dt.date

df_cuarentena.head()

,ID,Nombre,Estado,Alcance,Fecha de Inicio,Fecha de Término,Código CUT Comuna,Detalle,Superficie en m2,Perímetro en m,fecha_inicio,fecha_termino
0,1,Arica,Histórica,Área Urbana Completa,2020-04-17 02:00:00,2020-05-16 02:00:00,15101,Corresponde al Radio Urbano de la Ciudad de Ar...,2.805042e+07,48722.300013,2020-04-17,2020-05-16
1,2,Santiago Norte,Histórica,Sector Específico,2020-04-13 09:00:00,2020-05-06 02:00:00,13101,Corresponde a la extensión de la cuarentena en...,1.993922e+07,21599.351343,2020-04-13,2020-05-06
2,3,Puente Alto Poniente,Histórica,Sector Específico,2020-04-10 02:00:00,2020-05-09 02:00:00,13201,Corresponde al sector delimitado por la Av. Co...,4.236427e+07,30428.051523,2020-04-10,2020-05-09
3,4,El Bosque,Activa,Comuna completa,2020-04-17 02:00:00,2020-06-27 02:00:00,13105,Aplicada a la totalidad de la comuna. Se incor...,2.062271e+07,18788.870751,2020-04-17,2020-06-27
4,5,San Bernardo Nororiente,Histórica,Sector Específico,2020-04-17 02:00:00,2020-05-09 02:00:00,13401,Corresponde al sector delimitado por los limit...,1.632622e+07,24929.145720,2020-04-17,2020-05-09


In [29]:
df_list = []
for i, row in df_cuarentena.iterrows():
    comuna = row['Código CUT Comuna']
    fecha_ini = row['fecha_inicio']
    fecha_end = row['fecha_termino']
    fechas = pd.date_range(start=fecha_ini,end=fecha_end)
    df_aux = pd.DataFrame(fechas)
    df_aux['comuna'] = comuna
    df_list.append(df_aux)
df_fechas = pd.concat(df_list)
df_fechas.drop_duplicates(inplace=True)
df_fechas.rename(columns={0: 'fecha'}, inplace=True)
df_fechas['cuarentena'] = 1
df_fechas.head()


,fecha,comuna,cuarentena
0,2020-04-17,15101,1
1,2020-04-18,15101,1
2,2020-04-19,15101,1
3,2020-04-20,15101,1
4,2020-04-21,15101,1


# Calculo cuarentena o no

In [30]:
mobility = mobility.merge(df_fechas, how='left', left_on='Fecha', right_on='fecha')

In [31]:
mobility['cuarentena'] = mobility['cuarentena'].fillna(0)
mobility.head()

,Region,Codigo region,Comuna,Codigo comuna,Superficie_km2,Poblacion,IM_interno,IM_externo,IM,Fecha,lagged_Fecha,fecha,comuna,cuarentena
0,Ñuble,16.0,Chillán,16101,477.58,198624.0,5.922157,1.878393,7.800551,2020-02-26,2020-02-18,NaT,NaN,0.0
1,Ñuble,16.0,Chillán,16101,477.58,198624.0,5.453929,1.877832,7.331762,2020-02-27,2020-02-19,NaT,NaN,0.0
2,Ñuble,16.0,Chillán,16101,477.58,198624.0,5.616649,1.986287,7.602936,2020-02-28,2020-02-20,NaT,NaN,0.0
3,Ñuble,16.0,Chillán,16101,477.58,198624.0,4.800831,1.883542,6.684373,2020-02-29,2020-02-21,NaT,NaN,0.0
4,Ñuble,16.0,Chillán,16101,477.58,198624.0,3.706828,1.611526,5.318354,2020-03-01,2020-02-22,NaT,NaN,0.0


In [32]:
mobility

,Region,Codigo region,Comuna,Codigo comuna,Superficie_km2,Poblacion,IM_interno,IM_externo,IM,Fecha,lagged_Fecha,fecha,comuna,cuarentena
0,Ñuble,16.0,Chillán,16101,477.58,198624.0,5.922157,1.878393,7.800551,2020-02-26,2020-02-18,NaT,NaN,0.0
1,Ñuble,16.0,Chillán,16101,477.58,198624.0,5.453929,1.877832,7.331762,2020-02-27,2020-02-19,NaT,NaN,0.0
2,Ñuble,16.0,Chillán,16101,477.58,198624.0,5.616649,1.986287,7.602936,2020-02-28,2020-02-20,NaT,NaN,0.0
3,Ñuble,16.0,Chillán,16101,477.58,198624.0,4.800831,1.883542,6.684373,2020-02-29,2020-02-21,NaT,NaN,0.0
4,Ñuble,16.0,Chillán,16101,477.58,198624.0,3.706828,1.611526,5.318354,2020-03-01,2020-02-22,NaT,NaN,0.0
5,Ñuble,16.0,Chillán,16101,477.58,198624.0,5.736676,1.806846,7.543522,2020-03-02,2020-02-23,NaT,NaN,0.0
6,Ñuble,16.0,Chillán,16101,477.58,198624.0,5.743787,1.715189,7.458976,2020-03-03,2020-02-24,NaT,NaN,0.0
7,Ñuble,16.0,Chillán,16101,477.58,198624.0,5.716081,1.674993,7.391074,2020-03-04,2020-02-25,NaT,NaN,0.0
8,Ñuble,16.0,Chillán,16101,477.58,198624.0,5.978386,1.655035,7.633421,2020-03-05,2020-02-26,NaT,NaN,0.0
9,Ñuble,16.0,Chillán,16101,477.58,198624.0,6.270257,1.808783,8.079041,2020-03-06,2020-02-27,NaT,NaN,0.0


### Ingresos

Estimación para áreas pequeñas (SAE) 2017

In [33]:
# min_desarrollo = "http://observatorio.ministeriodesarrollosocial.gob.cl/documentos/"


# ingresos = (pd.read_excel(min_desarrollo + 
#     "PLANILLA_Estimaciones_comunales_tasa_pobreza_por_ingresos_multidimensional_2017.xlsx",
#                         skiprows=2)
#            .rename(columns={"Código": "Codigo comuna"}))

In [34]:
#ESTA DATA SALE DEL ETL
ingresos = pd.read_pickle(r'data/data_por_comuna.pkl')

In [35]:
ingresos.head(2)

,comuna,ytotcorh,esc,Codigo comuna,Región,Nombre comuna,Número de personas en situación de pobreza por ingresos,Porcentaje de personas en situación de pobreza por ingresos 2017,Límite inferior,Límite superior,Metodología de Estimación
0,1101,1.231723e+06,13.566768,1101,I de Tarapacá,Iquique,7928,0.043836,0.036953,0.051435,SAE
1,1107,9.365696e+05,12.321213,1107,I de Tarapacá,Alto Hospicio,9847,0.077555,0.068831,0.086089,SAE


#### Education? other variables?

### Merge

In [36]:
mob_cod_comuna = set(mobility['Codigo comuna'].unique())
ingresos_cod_comuna = set(ingresos['Codigo comuna'].unique())
assert len(mob_cod_comuna - ingresos_cod_comuna) < 2 

AssertionError: 

In [37]:
df = pd.merge(mobility, ingresos, how='left', on='Codigo comuna')
df['density'] = df['Poblacion'] / df['Superficie_km2']

In [38]:
df_grouped = df.groupby(['Codigo comuna', 'Nombre comuna', 'cuarentena']).mean()[['ytotcorh', 'esc', 'IM_interno']]
df_grouped

ytotcorh        esc  \
Codigo comuna Nombre comuna        cuarentena                            
1101          Iquique              0.0         1.231723e+06  13.566768   
                                   1.0         1.231723e+06  13.566768   
1107          Alto Hospicio        0.0         9.365696e+05  12.321213   
                                   1.0         9.365696e+05  12.321213   
1401          Pozo Almonte         0.0         9.683889e+05  11.426825   
                                   1.0         9.683889e+05  11.426825   
1402          Camiña               0.0         7.669497e+05   8.847682   
                                   1.0         7.669497e+05   8.847682   
1404          Huara                0.0         7.714938e+05   9.796911   
                                   1.0         7.714938e+05   9.796911   
1405          Pica                 0.0         9.529053e+05  11.895611   
                                   1.0         9.529053e+05  11.895611   
2101          Antofagasta          0.0         1.391606e+06  13.683464   
                                   1.0         1.391606e+06  13.683464   
2102          Mejillones           0.0         1.062425e+06  12.079603   
                                   1.0         1.062425e+06  12.079603   
2103          Sierra Gorda         0.0         9.329144e+05  12.280206   
                                   1.0         9.329144e+05  12.280206   
2104          Taltal               0.0         1.010900e+06  10.326032   
                                   1.0         1.010900e+06  10.326032   
2201          Calama               0.0         1.336860e+06  12.057928   
                                   1.0         1.336860e+06  12.057928   
2203          San Pedro de Atacama 0.0         9.022541e+05  10.617880   
                                   1.0         9.022541e+05  10.617880   
2301          Tocopilla            0.0         8.784803e+05  12.207501   
                                   1.0         8.784803e+05  12.207501   
2302          María Elena          0.0         1.024332e+06  12.298578   
                                   1.0         1.024332e+06  12.298578   
3101          Copiapó              0.0         1.207895e+06  12.920301   
                                   1.0         1.207895e+06  12.920301   
...                                                     ...        ...   
16107         Quillón              0.0         7.090576e+05  10.939865   
                                   1.0         7.090576e+05  10.939865   
16108         San Ignacio          0.0         4.858291e+05   9.671656   
                                   1.0         4.858291e+05   9.671656   
16109         Yungay               0.0         5.941331e+05  11.392852   
                                   1.0         5.941331e+05  11.392852   
16201         Quirihue             0.0         6.324886e+05  11.984437   
                                   1.0         6.324886e+05  11.984437   
16202         Cobquecura           0.0         5.651003e+05  10.466216   
                                   1.0         5.651003e+05  10.466216   
16203         Coelemu              0.0         7.313072e+05  11.387864   
                                   1.0         7.313072e+05  11.387864   
16204         Ninhue               0.0         5.049893e+05   8.837029   
                                   1.0         5.049893e+05   8.837029   
16205         Portezuelo           0.0         5.785400e+05   9.989913   
                                   1.0         5.785400e+05   9.989913   
16206         Ránquil              0.0         6.890595e+05  10.993036   
                                   1.0         6.890595e+05  10.993036   
16207         Treguaco             0.0         5.403878e+05   9.588774   
                                   1.0         5.403878e+05   9.588774   
16301         San Carlos           0.0         7.246347e+05  11.624720   
                                   1.0         7.246347

In [39]:
df_new = df_grouped.reset_index()
df_new.set_index(['Codigo comuna', 'Nombre comuna'], inplace=True)

df_antes = df_new[df_new.cuarentena == 0]
df_despues = df_new[df_new.cuarentena == 1]

df_new_new = df_antes.merge(df_despues, left_index = True, right_index = True)

df_new_new['change'] = df_new_new['IM_interno_y'] / df_new_new['IM_interno_x']
df_new_new.head()

,,cuarentena_x,ytotcorh_x,esc_x,IM_interno_x,cuarentena_y,ytotcorh_y,esc_y,IM_interno_y,change
Codigo comuna,Nombre comuna,,,,,,,,,
1101,Iquique,0.0,1.231723e+06,13.566768,7.712431,1.0,1.231723e+06,13.566768,5.345024,0.693040
1107,Alto Hospicio,0.0,9.365696e+05,12.321213,4.748399,1.0,9.365696e+05,12.321213,3.575572,0.753006
1401,Pozo Almonte,0.0,9.683889e+05,11.426825,0.924257,1.0,9.683889e+05,11.426825,0.685985,0.742202
1402,Camiña,0.0,7.669497e+05,8.847682,0.638725,1.0,7.669497e+05,8.847682,0.601366,0.941509
1404,Huara,0.0,7.714938e+05,9.796911,3.393399,1.0,7.714938e+05,9.796911,1.521684,0.448425


In [24]:
ax = sns.scatterplot(x="change", y="ytotcorh_x", 
                     data=df_new_new)
ax.set_title("Comunas en Chile \n % de cambio de movilidad vs porc pobreza")

AttributeError: module 'seaborn' has no attribute 'scatterplot'

## Plots

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5))
(df[df['Comuna'] == 'Las Condes'][['Comuna', 'Fecha', 'IM_interno',
                                            'IM_externo', 'IM']]
.set_index('Fecha')[['IM', 'IM_interno', 'IM_externo']]
 .rolling(8).mean().plot(ax=ax))
ax.set_title("Las Condes: Movilidad Interna y Externa")

In [ ]:
def comparar_dos_comunas(nom_comuna1: str, nom_comuna2: str, df_=df):
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    comuna1 = (df_[df_['Comuna'] == nom_comuna1].set_index('Fecha')
     ['IM_interno'].rolling(8).mean())

    (comuna1 / comuna1[9]).plot(ax=ax, label=f"{nom_comuna1} Int")

    comuna1 = (df_[df_['Comuna'] == nom_comuna1].set_index('Fecha')
     ['IM_externo'].rolling(8).mean())
    (comuna1 / comuna1[9]).plot(ax=ax, label=f"{nom_comuna1} Ext")

    comuna2 = (df_[df_['Comuna'] == nom_comuna2].set_index('Fecha')
     ['IM_interno'].rolling(8).mean())
    (comuna2 / comuna2[9]).plot(ax=ax, label=f"{nom_comuna2} Int")

    comuna2 = (df_[df_['Comuna'] == nom_comuna2].set_index('Fecha')
     ['IM_externo'].rolling(8).mean())
    (comuna2 / comuna2[9]).plot(ax=ax, label=f"{nom_comuna2} Ext")
    ax.legend()
    ax.set_title("Movilidad en dos comunas \n normalizado por tiempo inicial");
    
    return fig, ax

comparar_dos_comunas('Las Condes', 'La Granja');

In [ ]:
comparar_dos_comunas('Las Condes', 'Puente Alto');

### Scatter plots

+ `mobility_change`: what's the average internal mobility in the second half of the time period?
+ `delta_int_ext`: What the average % difference between internal and external mobility in the second half?

In [ ]:
def perc_diff(x, y):
    return 2*(x - y)/(x+y)

def get_mobility_change(comuna_mob):
    n_half = np.ceil(len(comuna_mob) / 2.).astype('int')
    if n_half > 4:
        second_half = np.mean(comuna_mob.values[n_half:])
        first_half = np.mean(comuna_mob.values[0:n_half])
        return perc_diff(second_half, first_half)
    else:
        return np.nan

def get_delta_int_ext(comuna):
    n_half = np.ceil(len(comuna) / 2.).astype('int')
    if n_half > 4:
        mob_int = np.mean(comuna['IM_interno'].values[n_half:])
        mob_ext = np.mean(comuna['IM_externo'].values[n_half:])
        return perc_diff(mob_int, mob_ext)
    else:
        return np.nan

def create_changes_df(df_):
    mobility_change = (df_.groupby("Codigo comuna")
                       ['IM_interno'].apply(get_mobility_change))
    mobility_change.name = 'mobility_change'
    delta_int_ext = (df_.groupby("Codigo comuna")
                       .apply(get_delta_int_ext))
    delta_int_ext.name = 'delta_int_ext'

    porc_pobreza = (df_.groupby('Codigo comuna')
        ['Porcentaje de personas en situación de pobreza por ingresos 2017']
                   .mean())
    porc_pobreza.name = "porc_pobreza"

    changes = pd.concat([mobility_change, delta_int_ext,
                        porc_pobreza], axis=1)
    return changes

changes_country = create_changes_df(df)

In [ ]:
ax = sns.scatterplot(x="mobility_change", y="porc_pobreza", 
                     data=changes_country)
ax.set_title("Comunas en Chile \n % de cambio de movilidad vs porc pobreza")

In [ ]:
ax = sns.scatterplot(x="delta_int_ext", y="porc_pobreza",
                     data=changes_country)
ax.set_title("Comunas en Chile \n % de diff entre movilidad externa e interna vs porc pobreza")

In [ ]:
metropolitana = df.query("Region == 'Metropolitana de Santiago' ")

changes_metropolitana = create_changes_df(metropolitana)

In [ ]:
type_ = "regresion"
if type_ == "regression":
    plot_function = sns.scatterplot
else:
    plot_function = sns.regplot
    
ax = plot_function(x="porc_pobreza", y="mobility_change", 
                     data=changes_metropolitana)
ax.set_title("Comunas en RM \n % de cambio de movilidad vs porc pobreza")

In [ ]:
ax = plot_function(x="delta_int_ext", y="porc_pobreza",
                     data=changes_metropolitana)
ax.set_title("Comunas en RM \n % de diff entre movilidad externa e interna vs porc pobreza")

In [ ]:
metropolitana.Comuna.nunique()